In [1]:
# 读取数据
import pandas as pd

# 读取数据的时候，可以把重复的和空值去掉
data_df = pd.read_csv('./data/result_Weibo_with_label.csv', 
                      encoding='utf8').dropna().drop_duplicates(['content'], keep='first')

stopwords = pd.read_csv('./data/data_stopwords.txt', index_col=False, sep='\t', 
                       quoting=3, names=['stopword'], encoding='utf8')
stopwords = stopwords.append(pd.read_csv('./data/data_stopwords_Weibo.txt', index_col=False, sep='\t', 
                                         quoting=3, names=['stopword'], encoding='utf8'))
stopwords.head()

,stopword
0,!
1,""""
2,#
3,$
4,%


In [3]:
# 数据预处理
# 1、分词
import jieba

contents = data_df['content'].values.tolist()    # 变成列表好操作一点
contents_segment = []
for content in contents:
    content_segment = jieba.lcut(content)
    if len(content_segment) > 1 and content_segment != '\t\r':
        contents_segment.append(content_segment)
%pprint        
contents_segment[:3][:3]

Pretty printing has been turned OFF


[['全球', '每年', '生产', '近', '1000', '亿件', '衣服', '，', '但', '最终', '85%', '的', '纺织品', '都', '要', '被', '丢弃', '和', '填埋', '，', '可', '持续', '的', '时尚', '行业', '将', '为', '地球', '环境', '带来', '积极', '转变', '。', '在', '<', 'a', ' ', 'target', '=', '"', '_', 'blank', '"', ' ', 'render', '=', '"', 'ext', '"', ' ', 'suda', '-', 'uatrack', '=', '"', 'key', '=', 'topic', '_', 'click', '&', 'amp', ';', 'value', '=', 'click', '_', 'topic', '"', ' ', 'class', '=', '"', 'a', '_', 'topic', '"', ' ', 'extra', '-', 'data', '=', '"', 'type', '=', 'topic', '"', ' ', 'href', '=', '"', '/', '/', 's', '.', 'weibo', '.', 'com', '/', 'weibo', '?', 'q', '=', '%', '23%', 'E5%', '9C%', 'B0%', 'E7%', '90%', '83%', 'E8%', 'A1%', 'A8%', 'E7%', '99%', 'BD%', 'E6%', '97%', 'A5%', '23', '&', 'amp', ';', 'from', '=', 'default', '"', '>', '#', '地球', '表白', '日', '#', '<', '/', 'a', '>', ' ', '与', '我', '和', '<', 'a', ' ', 'target', '=', '"', '_', 'blank', '"', ' ', 'render', '=', '"', 'ext', '"', ' ', 'extra', '-', 'data', '=', '"', 'type',

In [7]:
len(contents_segment[:3][:3])

3

In [9]:
# 2、去除停用词和非中文词
def drop_stopwords(texts, stopwords):
    clean_texts = []
    for text in texts:
        clean_words = []
        for word in text:
            if word in stopwords or not '\u4e00' <= word <= '\u9fff':
                continue
            else:
                clean_words.append(word)
        
        clean_texts.append(clean_words)
        
    return clean_texts

clean_contents = [' '.join(text) for text in drop_stopwords(contents_segment, stopwords)]
data_df['content'] = clean_contents
data_df.head()

,content,label
0,全球 每年 生产 近 亿件 衣服 但 最终 的 纺织品 都 要 被 丢弃 和 填埋 可 持续...,社会
1,从 幸福 到 灾难 只 需要 一瞬间 摄影师 在 黎巴嫩 首都 贝鲁特 大 爆炸 前 正在 ...,社会
2,湖南 高一 新生 军训 时 死亡 湖南 高一 新生 军训 时 晕倒 身亡 教育局 通报 系因...,社会
3,上海 首例 养犬人 遗弃 犬 只案 警方 罚狗 主人 五百元 吊销 犬证 月 日 上午 在 ...,社会
4,男子 携子 威胁 妻子 并称 我 只 打过 她 次 都 没进 医院 月 日 家住 四川省 乐...,社会


In [20]:
# 划分数据集
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data_df['content'].values, 
                                                   data_df['label'].values, random_state=1)
y_train[:3]

array(['健康', '动漫', '游戏'], dtype=object)

In [23]:
# 构建词向量
from sklearn.feature_extraction.text import TfidfVectorizer
vel_model = TfidfVectorizer()

x_train_vel = vel_model.fit_transform(x_train).toarray()
x_test_vel = vel_model.transform(x_test).toarray()      # 不能两个都fit，只用一个训练就可以了
# 不然每个词典不一样，

In [24]:
# 使用随机梯度分类器
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier()
clf.fit(x_train_vel, y_train)
clf.score(x_test_vel, y_test)

0.8071604938271605